# 🎤 CLARISSA Voice Input Showcase

**Talk to Your Reservoir Simulation**

This notebook demonstrates CLARISSA's voice interface for controlling reservoir simulations through natural language commands.

---

## 🔄 Workflow

| Step | Where | What |
|------|-------|------|
| 1️⃣ | **Web Demo** | Record your voice command |
| 2️⃣ | **This Notebook** | Upload + Transcribe + Process |

---

## 🚀 Step 1: Record Your Voice

👉 **[Open Voice Demo](https://irena-40cc50.gitlab.io/demos/voice-demo.html)** (opens in new tab)

1. Click **"Start Recording"**
2. Speak your command (e.g., *"show permeability distribution"*)
3. Click **"Stop"**
4. Click **"Download WAV"**
5. Come back here and upload the file

---

In [ ]:
# Setup (run once)
!pip install -q openai

import os
from IPython.display import display, HTML, Audio
from google.colab import files

print("✅ Ready! Upload your audio file below.")

---

## 📤 Step 2: Upload Your Recording

In [ ]:
# Upload your recorded audio file
print("📂 Select your audio file (WAV, WebM, MP3, etc.)")
print()

uploaded = files.upload()

if uploaded:
    filename = list(uploaded.keys())[0]
    audio_bytes = uploaded[filename]
    
    print()
    print(f"✅ Uploaded: {filename} ({len(audio_bytes):,} bytes)")
    print()
    print("🔊 Playback:")
    display(Audio(audio_bytes, autoplay=False))
else:
    print("❌ No file uploaded")
    audio_bytes = None

---

## 📝 Step 3: Transcribe with Whisper

Enter your OpenAI API key to transcribe the audio:

In [ ]:
# Transcribe the uploaded audio
import tempfile
from getpass import getpass

# Get API key
if not os.environ.get('OPENAI_API_KEY'):
    os.environ['OPENAI_API_KEY'] = getpass("OpenAI API Key: ")

if audio_bytes:
    from openai import OpenAI
    client = OpenAI()
    
    # Save to temp file
    ext = filename.split('.')[-1] if '.' in filename else 'wav'
    with tempfile.NamedTemporaryFile(suffix=f'.{ext}', delete=False) as f:
        f.write(audio_bytes)
        temp_path = f.name
    
    print("🔄 Transcribing...")
    
    with open(temp_path, 'rb') as audio_file:
        transcript = client.audio.transcriptions.create(
            model="whisper-1",
            file=audio_file,
            language="en",
            prompt="Reservoir simulation commands: permeability, porosity, pressure, saturation, water cut, oil rate, injection, production, layers, wells, grid"
        )
    
    os.unlink(temp_path)
    
    print()
    print("═" * 50)
    print("📝 TRANSCRIPT")
    print("═" * 50)
    print()
    print(f'   "{transcript.text}"')
    print()
    
    # Store for next cell
    command_text = transcript.text
else:
    print("❌ No audio to transcribe. Run the upload cell first.")
    command_text = None

---

## 🧠 Step 4: Parse Intent

Analyze the voice command to extract the user's intent:

In [ ]:
# Parse the intent from the transcript
if command_text:
    # Simple keyword-based intent detection
    text_lower = command_text.lower()
    
    # Define intents
    intents = []
    
    # Property queries
    if any(w in text_lower for w in ['permeability', 'perm']):
        intents.append(('SHOW_PROPERTY', 'permeability'))
    if any(w in text_lower for w in ['porosity', 'poro']):
        intents.append(('SHOW_PROPERTY', 'porosity'))
    if 'pressure' in text_lower:
        intents.append(('SHOW_PROPERTY', 'pressure'))
    if any(w in text_lower for w in ['saturation', 'sat']):
        intents.append(('SHOW_PROPERTY', 'saturation'))
    if any(w in text_lower for w in ['water cut', 'watercut']):
        intents.append(('SHOW_METRIC', 'water_cut'))
    if any(w in text_lower for w in ['oil rate', 'production rate']):
        intents.append(('SHOW_METRIC', 'oil_rate'))
    
    # Actions
    if any(w in text_lower for w in ['run', 'simulate', 'execute']):
        intents.append(('RUN_SIMULATION', None))
    if any(w in text_lower for w in ['stop', 'cancel', 'abort']):
        intents.append(('STOP_SIMULATION', None))
    
    # Questions
    if any(w in text_lower for w in ['what is', 'what are', 'how much', 'how many']):
        intents.append(('QUERY', 'general'))
    
    print("═" * 50)
    print("🧠 INTENT ANALYSIS")
    print("═" * 50)
    print()
    print(f"Input: \"{command_text}\"")
    print()
    
    if intents:
        print("Detected intents:")
        for intent_type, intent_value in intents:
            if intent_value:
                print(f"  • {intent_type}: {intent_value}")
            else:
                print(f"  • {intent_type}")
    else:
        print("  ⚠️ No specific intent detected")
        print("  → Would pass to LLM for interpretation")
    
    print()
    print("─" * 50)
    print("💡 In production, this would trigger CLARISSA actions")
else:
    print("❌ No transcript available. Run the transcription cell first.")

---

## 🔗 Resources

| Resource | Link |
|----------|------|
| 🎤 Voice Demo | [irena-40cc50.gitlab.io/demos/voice-demo.html](https://irena-40cc50.gitlab.io/demos/voice-demo.html) |
| 🎬 Recording Suite | [irena-40cc50.gitlab.io/demos/demo-recording-suite.html](https://irena-40cc50.gitlab.io/demos/demo-recording-suite.html) |
| 📚 CLARISSA Docs | [irena-40cc50.gitlab.io](https://irena-40cc50.gitlab.io/) |
| 💻 Source Code | [GitLab Repository](https://gitlab.com/wolfram_laube/blauweiss_llc/irena) |

---

## 📋 Example Commands to Try

- *"Show me the permeability distribution"*
- *"What is the current water cut?"*
- *"Display pressure in layer 3"*
- *"Run the simulation"*
- *"What are the oil production rates?"*